<a href="https://colab.research.google.com/github/danielgoldwyn/Merge-CSV-EXCEL-gdrive/blob/main/Merge_CSV_XLXS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook can be used to merge .csv or .xlsx files hosted in your google drive folder

In [ ]:
# import dependencies

import glob
import pandas as pd
import openpyxl
import chardet
import csv
from google.colab import files
from google.colab import drive

In [ ]:
# mount your google drive so that colab may access and find the folder containing files to be merged
# you will have to grant colab permission to access your google drive.
# IMPORTANT - ensure that the google ID used to log into colab is the same google ID in which the drive folder is stored in

drive.mount('/content/drive', force_remount=True)

In [ ]:
# set path of the folder in your google drive containing .csv or .xlsx
# IMPORTANT - if the folder has been shared to you by someone else - you need to create a shortcut of that folder into your drive for colab to find it. right click on the shared folder in google drive and click on create shortcut.

path = '/content/drive/MyDrive/set-folder-path-here'

In [ ]:
# read all filename in the folder specified above

file_list = glob.glob(path + "/*.csv")
# above example is to only read filenames that are .csv filetype. change it to "/*.xlsx" if you want to read filenames of .xlsx filetypes
# if you are confident that only the files that you want to read are in the folder you may also choose to replace "/*.csv" to "/*.*" while will read filenames of all files in folder
# "/*.*" can also be used when you are merging a mix of .xlsx and .csv files but you need to ensure that there are not any other types of file in the folder

In [ ]:
# print out number of files read to confirm total number of files that you are going to merge

print(len(file_list))

In [ ]:
# read all data in the files

# create an empty list to store data of each file read
excl_list = []

for file in file_list:
    # print filename so that if any error happens the last file in the output will be the one with problem
    print(file)

    # open the file
    with open(file, 'rb') as rawdata:
      # pass first 10,000 bytes of rawdata to chardet.detect to determine file encoding
      # this will be output below the filename in case of any errors to help with debugging
      print(chardet.detect(rawdata.read(10000)))

    # read the data and append it to excl_list
    try:
      # using pandas .read_excel funtion to read the .xlsx file
      excl_list.append(pd.read_excel(file))
      # if the .xlsx file has a last row with data that you do not want to keep in the merged file (example a totals cell), use the skipfooter flag as below
      # excl_list.append(pd.read_excel(file, skipfooter=1))
      # skipfooter=1 tells the script to ignore the last line. if this value is changed to 2, script will ignore the last two lines etc..

    except:
      # using pandas .read_csv funtion to read the .csv file
      excl_list.append(pd.read_csv(file))
      # if the .csv file has a last row with example total that you do not want to keep in the merged file use the skipfooter flag as below
      # excl_list.append(pd.read_csv(file, skipfooter=1))
      # skipfooter=1 tells the script to ignore the last line. if this value is changed to 2, script will ignore the last two lines etc..


In [ ]:
# merge all data that has been read using pandas

# create an pandas dataframe
excl_merged = pd.DataFrame()

# merge data
for excl_file in excl_list:
  # use pandas .concat to merge all data
  excl_merged = pd.concat(excl_list, ignore_index=True)
  # above pandas function will automatically generate a new first column with index 0-n. To avoid this use the ignore_index flag
  # excl_merged = excl_merged.append(excl_file, ignore_index=True)

In [ ]:
# create a .csv file of the merged data and download it to the local storage

excl_merged.to_csv('output.csv', encoding = 'utf-8')
files.download('output.csv')